In [1]:
# Import Necessary Libraries
import pydocumentdb
from pydocumentdb import document_client
from pydocumentdb import documents
import datetime

# Configuring the connection policy (allowing for endpoint discovery)
connectionPolicy = documents.ConnectionPolicy()
connectionPolicy.EnableEndpointDiscovery 
connectionPolicy.PreferredLocations = ["Central US", "East US 2", "Southeast Asia", "Western Europe","Canada Central"]

# Set keys to connect to Cosmos DB 
masterKey = 'ZvHF3rrP2qaNQ6Yri5rjkKpVTBPNTq2iJ1k28EMBSne4DkSXscNTJzRGG4QZktBfREgfOmSqKCzYGPzqDqZ3sw==' 
host = 'https://cs489cosmossql.documents.azure.com:443/'
client = document_client.DocumentClient(host, {'masterKey': masterKey}, connectionPolicy)

# Configure Database and Collections
databaseId = 'cralSpectra'
collectionId = 'experimentalSpectra'

# Configurations the Cosmos DB client will use to connect to the database and collection
dbLink = 'dbs/' + databaseId
collLink = dbLink + '/colls/' + collectionId

# Set query parameter
#querystr = "SELECT c.City FROM c WHERE c.State='WA'"
querystr = "SELECT * FROM experimentalSpectra f WHERE f.Compound = 'Unknown'"

# Query documents
query = client.QueryDocuments(collLink, querystr, options=None, partition_key=None)

# Query for partitioned collections
# query = client.QueryDocuments(collLink, querystr, options= { 'enableCrossPartitionQuery': True }, partition_key=None)

# Push into list `elements`
elements = list(query)

bcontinue = False
# Create `df` Spark DataFrame from `elements` Python list
if len(elements) > 0:
  bcontinue = True
  df_exp = spark.createDataFrame(elements)


In [2]:
if bcontinue == True:
    a_exp = df_exp.rdd.map(lambda p: p.PeakList.split(';')).collect()
    b_exp = df_exp.rdd.map(lambda x: x.Compound).collect()
    id_exp = df_exp.rdd.map(lambda x: x._self).collect()
    bcontinue = True;

In [3]:
if bcontinue == True:
    import pandas as pd
    from pyspark.sql import functions as F
    from pyspark.sql.types import *

    mz_arr = []
    intensity_arr = []
    sample_name = []
    sample_id = []
    
    print(len(a_exp))
    
    for i in range(0,len(a_exp)):
    #print(i)
        for item in a_exp[i]:
            mz, intensity = item.split(':')
            mz_arr.append(int(mz))
            intensity_arr.append(int(intensity))
            #print(id_exp[i])
            sample_id.append(id_exp[i])
        
        
    #added so during pivot, data will be all in vectors of sample length
    for i in range(0, 1761):
        mz_arr.append(i)
        intensity_arr.append(0)
        sample_name.append("reference")
        sample_id.append("none")
  
    df_exp = pd.DataFrame({'sample_id': sample_id,'mz':mz_arr, 'intensity':intensity_arr})


    #Create PySpark DataFrame Schema
    p_schema = StructType([StructField('intensity',IntegerType(),True),StructField('mz',IntegerType(),True),StructField('sample_id',StringType(),True)])


    #Create Spark DataFrame from Pandas
    df_spark2_exp = sqlContext.createDataFrame(df_exp, p_schema)

    reshaped_df_exp = df_spark2_exp.groupby('sample_id').pivot('mz').max('intensity').fillna(0).filter(df_spark2_exp.sample_id != "none")
    
    #reshaped_df_exp.show()

    #reshaped_df_exp.write.saveAsTable("spectra_pivotted_exp7")


In [4]:
#collect took 16.32 seconds
if bcontinue == True:
    ref_samplenames = spark.sql("SELECT sample_name FROM spectra_pivotted").collect()
    #ref_samplenames_collect = spark.sql("SELECT sample_name FROM spectra_pivotted").collect()
    ref_intensity = spark.sql("SELECT * FROM spectra_pivotted").drop("sample_name")
    bcontinue = True

In [5]:
if bcontinue == True:
    import math
    import numpy as np
    from numpy import array
    mzs = np.arange(1761)
    arr_mzs = array(mzs)
    score = []
    j = 0
    exp_intensity = reshaped_df_exp.drop("sample_id").collect()
    exp_id = reshaped_df_exp.select("sample_id").collect()
    
    def dotproduct(row):
        result = np.multiply(np.dot(row, arr_mzs), exp_demoninator)
        return result
    
    def numerator(row):
        result = np.dot(np.sqrt(np.multiply(row, array(exp_intensity))),arr_mzs)
        result = result * result
        return result

    exp_demoninator = np.dot(exp_intensity,arr_mzs )
    #print("exp_demon")
    #print(exp_demoninator)
    #denominators = np.multiply(ref_demoninator, exp_demoninator)
    temp_denom = ref_intensity.rdd.map(dotproduct)
    denominators = temp_denom.collect()
    #print("denominators")
    #print(denominators[0])
    numerators = array(ref_intensity.rdd.map(numerator).collect())
    #print(len(denominators))
    #print(denominators[0])
    #print(len(numerators))
    #print(numerators[0])
    val = np.divide(numerators, denominators)
    #print(type(val))
    #print(len(val))
    #print(val[0])    
    bcontinue = True


In [6]:
if bcontinue == True:
    new_dict = {}  
    for j in range(0, len(exp_intensity)):
        arr_sample_max_ids = val[:,j]
        sample_idx = np.argpartition(arr_sample_max_ids, -5)[-5:]
        #sample_idx = np.argpartition(val, -5)[-5:]
        print(sample_idx)
        strResult = (np.take(ref_samplenames, sample_idx))
        strId = exp_id[j][0]
        new_dict.update({strId: strResult})

    print(new_dict)
    bcontinue = True

In [7]:
# Import Necessary Libraries
if bcontinue == True:
    import pydocumentdb
    from pydocumentdb import document_client
    from pydocumentdb import documents
    import datetime

    # Configuring the connection policy (allowing for endpoint discovery)
    connectionPolicy = documents.ConnectionPolicy()
    connectionPolicy.EnableEndpointDiscovery 
    connectionPolicy.PreferredLocations = ["Central US", "East US 2", "Southeast Asia", "Western Europe","Canada Central"]

    # Set keys to connect to Cosmos DB 
    masterKey = 'ZvHF3rrP2qaNQ6Yri5rjkKpVTBPNTq2iJ1k28EMBSne4DkSXscNTJzRGG4QZktBfREgfOmSqKCzYGPzqDqZ3sw==' 
    host = 'https://cs489cosmossql.documents.azure.com:443/'
    client = document_client.DocumentClient(host, {'masterKey': masterKey}, connectionPolicy)

    # Configure Database and Collections
    databaseId = 'cralSpectra'
    collectionId = 'experimentalSpectra'

    # Configurations the Cosmos DB client will use to connect to the database and collection
    dbLink = 'dbs/' + databaseId
    collLink = dbLink + '/colls/' + collectionId

    for key, value in new_dict.items():
        id = key
        # use a SQL based query to get a bunch of documents
        querystr = "SELECT * FROM experimentalSpectra f WHERE f._self = '" + id + "'"

        # Query documents
        query = client.QueryDocuments(collLink, querystr, options=None, partition_key=None)

        # Query for partitioned collections
        # query = client.QueryDocuments(collLink, querystr, options= { 'enableCrossPartitionQuery': True }, partition_key=None)

        # Push into list `elements`
        elements = list(query)
        print(len(elements))
        doc= elements[0]

        doc["Compound"] = ' '.join(value)

        # Get the document link from attribute `_self`
        doc_link = doc['_self']
        # Modify the document
        # Replace the document via document link
        client.ReplaceDocument(doc_link, doc)
    